In [18]:
# import libraries
import robin_stocks.robinhood as r 
import pandas as pd
import numpy as np
import pyotp
import matplotlib.pyplot as plt
import datetime as dt

In [19]:
# Login robinhood

totp  = pyotp.TOTP("My2factorAppHere").now()
print("Current OTP:", totp)
r.login(username='mihir.argulkar@gmail.com',
         password='P%,fvd#C96XkCjE',
         expiresIn=120, mfa_code=totp)


# grabs our current stock portfolio
my_stocks = r.build_holdings()
# grab basic stats such as dividends and cash portfolio
user = r.build_user_profile()

# Define directory and filenames
export_dir = '/Users/mihirargulkar/robinhood-dashboard'
stock_orders_filename = 'stock_orders_Jan-20-2023.csv'
option_orders_filename = 'option_orders_Jan-20-2023.csv'

# grab history of sold stocks on the market
sold_stocks = r.export_completed_stock_orders(export_dir, stock_orders_filename)
# grab history of sold options on the market
sold_options = r.export_completed_option_orders(export_dir, option_orders_filename)

# create dataframes of our sold stocks and sold options
df_sold_stocks = pd.read_csv(stock_orders_filename)
df_sold_options = pd.read_csv(option_orders_filename)






Current OTP: 698950
Starting login process...
Found Additional pages.
Loading page 2 ...


In [20]:

option_orders = pd.read_csv('option_orders_Jan-20-2023.csv')
# change the order_created_at to datetime
option_orders['order_created_at'] = pd.to_datetime(option_orders['order_created_at'])
option_orders = option_orders.sort_values(by=['chain_symbol', 'expiration_date', 'order_created_at'], ascending=True)
option_orders = option_orders.drop(['order_type'], axis=1)

# Fill missing opening_strategy with closing_strategy, and vice versa
option_orders['opening_strategy'] = option_orders['opening_strategy'].fillna(option_orders['closing_strategy'])
option_orders['closing_strategy'] = option_orders['closing_strategy'].fillna(option_orders['opening_strategy'])
# Fix the creation of the 'id' column so that it is a string concatenation of the relevant columns, not a single string of the Series objects.
option_orders['id'] = (
    option_orders['chain_symbol'].astype(str) +
    option_orders['expiration_date'].astype(str) +
    option_orders['strike_price'].astype(str) +
    option_orders['option_type'].astype(str) +
    option_orders['opening_strategy'].astype(str) 
)

option_orders['id_leg'] = option_orders['direction'].astype(str) +  option_orders['id']

# If there are multiple rows with the same 'id_leg', drop the duplicates and increase the 'order_quantity' by the number of duplicates dropped
order_quantity = option_orders.groupby('id_leg').agg({'order_quantity': 'sum'}).reset_index()
option_orders['count'] = option_orders.groupby('id_leg')['id_leg'].transform('count')
option_orders['order_quantity'] = option_orders['order_quantity'] * option_orders['count']
option_orders = option_orders.drop(columns=['count'])


# # drop duplicate id_leg
option_orders = option_orders.drop_duplicates(subset=['id_leg'])




option_orders['closing_strategy'].value_counts()

closing_strategy
short_put            127
short_put_spread      28
short_call            27
long_call              9
short_call_spread      4
long_put               2
Name: count, dtype: int64

In [161]:
# Concise processing for short puts and calls
short_trades = option_orders[option_orders['opening_strategy'].isin(['short_put', 'short_call'])].copy()
short_trades = short_trades.rename(columns={'chain_symbol': 'symbol', 'option_type': 'type', 'order_quantity': 'quantity'})
short_trades = short_trades.groupby('id').agg({
    'opening_strategy': 'first',
    'symbol': 'first',
    'strike_price': 'first',
    'expiration_date': 'first',
    'type': 'first',
    'quantity': 'first',
    'price': 'first',
    'direction': list
})

def get_exit_price(id_):
    group = option_orders[option_orders['id'] == id_]
    if (group['direction'] == 'debit').any():
        return group['price'].iloc[-1]
    return 0

short_trades['entry_price'] = short_trades['price']
short_trades['exit_price'] = short_trades.index.to_series().map(get_exit_price)
short_trades['pnl'] = (short_trades['entry_price'] - short_trades['exit_price']) * short_trades['quantity'] * 100
short_trades['cumulative_pnl'] = short_trades['pnl'].cumsum()
short_trades['collateral'] = short_trades['strike_price'] * short_trades['quantity'] * 100
short_trades['roi'] = ((short_trades['pnl'] / short_trades['collateral']) * 100).round(2)
short_trades = short_trades[[
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]]


In [160]:
# Concise processing for long puts and calls
long_trades = option_orders[option_orders['opening_strategy'].isin(['long_put', 'long_call'])].copy()
long_trades = long_trades.rename(columns={'chain_symbol': 'symbol', 'option_type': 'type', 'order_quantity': 'quantity'})
long_trades = long_trades.groupby('id').agg({
    'opening_strategy': 'first',
    'symbol': 'first',
    'strike_price': 'first',
    'expiration_date': 'first',
    'type': 'first',
    'quantity': 'first',
    'price': 'first',
    'direction': list
})

def get_exit_price(id_):
    group = option_orders[option_orders['id'] == id_]
    if (group['direction'] == 'credit').any():
        return group['price'].iloc[-1]
    return 0

long_trades['entry_price'] = long_trades['price']
long_trades['exit_price'] = long_trades.index.to_series().map(get_exit_price)
long_trades['pnl'] = (long_trades['exit_price'] - long_trades['entry_price']) * long_trades['quantity'] * 100
long_trades['cumulative_pnl'] = long_trades['pnl'].cumsum()
long_trades['collateral'] = long_trades['entry_price'] * long_trades['quantity'] * 100
long_trades['roi'] = ((long_trades['pnl'] / long_trades['collateral']) * 100).round(2)
long_trades = long_trades[[
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]]
long_trades.shape

(6, 12)

In [159]:
# Short spreads (put/call): concise processing
ss_trades = option_orders[option_orders['opening_strategy'].isin(['short_put_spread', 'short_call_spread'])].copy()
ss_trades = ss_trades.rename(columns={'chain_symbol': 'symbol', 'order_quantity': 'quantity', 'option_type': 'type'})
group_idx = np.arange(len(ss_trades)) // 4
ss_trades['avg_strike'] = ss_trades['strike_price'].astype(float).groupby(group_idx).transform('mean')
ss_trades['id'] = (
    ss_trades['symbol'].astype(str) + "_" +
    ss_trades['avg_strike'].astype(str) + "_" +
    ss_trades['expiration_date'].astype(str) + "_" +
    ss_trades['type'].astype(str) + "_" +
    ss_trades['opening_strategy'].astype(str)
)
ss_trades = ss_trades.sort_values(['id', 'direction'])
group_cols = ['opening_strategy', 'symbol', 'avg_strike', 'expiration_date', 'type', 'id']
agg = {
    'strike_price': lambda x: list(dict.fromkeys(x)),
    'quantity': 'mean',
    'price': lambda x: list(dict.fromkeys(x)),
    'processed_quantity': 'mean',
    'direction': lambda x: list(dict.fromkeys(x))
}
ss_trades = ss_trades.groupby(group_cols, as_index=False).agg(agg)

def entry_exit(prices):
    if len(prices) == 2: return prices[0], prices[1]
    if len(prices) == 4: return prices[0] - prices[1], prices[2] - prices[3]
    return prices[0], prices[-1]

ss_trades['entry_price'], ss_trades['exit_price'] = zip(*ss_trades['price'].map(entry_exit))
ss_trades = ss_trades.set_index('id')

ss_trades['pnl'] = (ss_trades['entry_price'] - ss_trades['exit_price']) * 100 * ss_trades['quantity']
ss_trades['cumulative_pnl'] = ss_trades['pnl'].cumsum()
ss_trades['collateral'] = ss_trades.apply(
    lambda r: abs(float(r['strike_price'][0]) - float(r['strike_price'][1])) * 100 * r['quantity']
    if len(r['strike_price']) >= 2 else 0, axis=1
)
ss_trades['roi'] = (ss_trades['pnl'] / ss_trades['collateral'] * 100).round(2)

ss_trades = ss_trades[[
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'price', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]]

ss_trades


,opening_strategy,symbol,strike_price,expiration_date,type,quantity,price,entry_price,exit_price,pnl,cumulative_pnl,collateral,roi
id,,,,,,,,,,,,,
AAPL_218.75_2025-08-01_call_short_call_spread,short_call_spread,AAPL,"[217.5, 220.0]",2025-08-01,call,1.0,"[0.74, 0.57]",0.74,0.57,17.0,17.0,250.0,6.8
AMD_131.0_2025-07-18_put_short_put_spread,short_put_spread,AMD,"[132.0, 130.0]",2025-07-18,put,1.0,"[0.54, 0.42]",0.54,0.42,12.0,29.0,200.0,6.0
GOOGL_162.5_2025-07-18_put_short_put_spread,short_put_spread,GOOGL,"[160.0, 165.0]",2025-07-18,put,1.0,"[1.27, 0.76]",1.27,0.76,51.0,80.0,500.0,10.2
GOOGL_168.75_2025-07-18_put_short_put_spread,short_put_spread,GOOGL,"[170.0, 167.5]",2025-07-18,put,1.0,"[0.48, 0.34]",0.48,0.34,14.0,94.0,250.0,5.6
HOOD_87.5_2025-07-18_put_short_put_spread,short_put_spread,HOOD,"[88.0, 87.0]",2025-07-18,put,1.0,"[0.32, 0.13]",0.32,0.13,19.0,113.0,100.0,19.0
NVDA_147.0_2025-07-18_put_short_put_spread,short_put_spread,NVDA,"[148.0, 146.0]",2025-07-18,put,1.0,"[0.43, 0.32]",0.43,0.32,11.0,124.0,200.0,5.5
SPY_589.0_2025-06-20_put_short_put_spread,short_put_spread,SPY,"[590.0, 588.0]",2025-06-20,put,1.0,"[0.24, 0.03]",0.24,0.03,21.0,145.0,200.0,10.5
TSLA_303.75_2025-07-03_put_short_put_spread,short_put_spread,TSLA,"[302.5, 305.0]",2025-07-03,put,1.0,"[0.5, 0.32]",0.50,0.32,18.0,163.0,250.0,7.2


In [158]:
final_col_order = [
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]

all_cols = set(short_trades.columns) | set(long_trades.columns) | set(ss_trades.columns)
dfs = [short_trades, long_trades, ss_trades]
aligned = [df.reindex(columns=all_cols) for df in dfs]
combined_trades = pd.concat(aligned, axis=0).reindex(columns=final_col_order)
if 'pnl' in combined_trades:
    combined_trades['cumulative_pnl'] = combined_trades['pnl'].cumsum()
combined_trades


,opening_strategy,symbol,strike_price,expiration_date,type,quantity,entry_price,exit_price,pnl,cumulative_pnl,collateral,roi
id,,,,,,,,,,,,
AAL2025-07-1811.0putshort_put,short_put,AAL,11.0,2025-07-18,put,1.0,0.20,0.05,15.0,15.0,1100.0,1.36
ACHR2025-06-139.0putshort_put,short_put,ACHR,9.0,2025-06-13,put,1.0,0.18,0.04,14.0,29.0,900.0,1.56
ASTS2024-06-078.0putshort_put,short_put,ASTS,8.0,2024-06-07,put,2.0,0.15,0.00,30.0,59.0,1600.0,1.88
ASTS2024-06-149.0putshort_put,short_put,ASTS,9.0,2024-06-14,put,2.0,0.10,0.00,20.0,79.0,1800.0,1.11
ASTS2024-06-2110.5putshort_put,short_put,ASTS,10.5,2024-06-21,put,1.0,0.20,0.00,20.0,99.0,1050.0,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...
GOOGL_168.75_2025-07-18_put_short_put_spread,short_put_spread,GOOGL,"[170.0, 167.5]",2025-07-18,put,1.0,0.48,0.34,14.0,4277.0,250.0,5.60
HOOD_87.5_2025-07-18_put_short_put_spread,short_put_spread,HOOD,"[88.0, 87.0]",2025-07-18,put,1.0,0.32,0.13,19.0,4296.0,100.0,19.00
NVDA_147.0_2025-07-18_put_short_put_spread,short_put_spread,NVDA,"[148.0, 146.0]",2025-07-18,put,1.0,0.43,0.32,11.0,4307.0,200.0,5.50
